<a href="https://colab.research.google.com/github/SarathiSoundarya/MPI_Codes/blob/main/MPI_warmup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Hello World Program**

In [14]:
%%writefile helloworld.c
#include<stdio.h>
#include<mpi.h>
int main(int argc, char **argv)
{
  int rank, size; //size is the number of processors
  MPI_Init(&argc,&argv);
  MPI_Comm_rank(MPI_COMM_WORLD,&rank);
  MPI_Comm_size(MPI_COMM_WORLD,&size);
  printf("Helloo from rank %d of %d processes\newline",rank,size);
  MPI_Finalize();
  return 0;
}


Overwriting helloworld.c


In [15]:
%%shell
mpicc helloworld.c -o mpi_hello
mpirun --allow-run-as-root -np 4 ./mpi_hello

Helloo from rank 0 of 4 processes
Helloo from rank 1 of 4 processes
Helloo from rank 3 of 4 processes
Helloo from rank 2 of 4 processes
ewlineewlineewlineewline

**Lab Exercises**<br /><br />
**Question 1** Greetings example where all processes with ranks greater than 0 send their rank and size to the process with rank 0 for printing.

In [33]:
%%writefile ques1.c
#include<mpi.h>
#include<stdio.h>
#include<string.h>
#include<stdlib.h>
int main(int argc,char **argv)
{
  int rank,size,src,dest;
  int message[2];
  MPI_Status status;
  MPI_Init(&argc,&argv);
  MPI_Comm_rank(MPI_COMM_WORLD,&rank);
  MPI_Comm_size(MPI_COMM_WORLD,&size);
  if(size==1)
  {
    printf("This example requires more than one process to execute");
    MPI_Finalize();
    exit(0);
  }
  if (rank!=0)
  {
    message[0]=rank;
    message[1]=size;
    dest=0;   
    MPI_Send(message,2,MPI_INT,dest,0,MPI_COMM_WORLD);
  }
  else
  {
   for(src=1;src<size;src++)
   {
     MPI_Recv(message,2,MPI_INT,src,MPI_ANY_TAG,MPI_COMM_WORLD,&status);
     printf("Hello fromm process %d of %d\n",message[0],message[1]);
   } 
  }
 MPI_Finalize();
 return 0;
}

Overwriting ques1.c


In [34]:
%%shell
mpicc ques1.c -o ques1
mpirun --allow-run-as-root -np 4 ./ques1

Hello fromm process 1 of 4
Hello fromm process 2 of 4
Hello fromm process 3 of 4


**Question 2**
Create a vector of size 4 and assign random values on the root process. Write a funtion for Broadcast of vector from root process to the other processes. (run on 4 processes)

In [41]:
%%writefile ques2.c
#include<stdio.h>
#include<mpi.h>
#include<string.h>
#include<stdlib.h>
int main(int argc, char **argv)
{
 
int rank,size;
MPI_Init(&argc ,&argv ) ;
MPI_Comm_rank(MPI_COMM_WORLD,&rank) ; // get the rank of this process
MPI_Comm_size(MPI_COMM_WORLD,&size) ; //get the size of no of processors
int *a;
a=malloc(4*sizeof(int));
for(int i=0;i<4;i++)
{
a[i]=0;
}
if(rank==0)
{
for(int i=0;i<4;i++)
{
a[i]=i;
}
}
printf( "Pre−Broadcast: on %d process is \n" , rank ) ;
for(int i=0;i<4;i++)
{
printf(" %d, ",a[i]);
}
printf("\n");
MPI_Barrier(MPI_COMM_WORLD);
MPI_Bcast(a,4,MPI_INT,0,MPI_COMM_WORLD);
printf("\n");
printf( "Postbroadcast: on %d process is \n" , rank ) ;
for(int i=0;i<4;i++)
{
printf(" %d, ",a[i]);
}
printf("\n");
MPI_Finalize();
return 0;
}

Overwriting ques2.c


In [45]:
%%shell
mpicc ques2.c -o ques2
mpirun --allow-run-as-root -np 4 ./ques2

Pre−Broadcast: on 0 process is 
Pre−Broadcast: on 3 process is 
 0,  0,  0,  0, 
 0,  1,  2,  3, 
Pre−Broadcast: on 1 process is 
 0,  0,  0,  0, 
Pre−Broadcast: on 2 process is 
 0,  0,  0,  0, 

Postbroadcast: on 0 process is 

Postbroadcast: on 1 process is 
 0,  1,  2,  3, 

Postbroadcast: on 2 process is 
 0,  1,  2,  3, 
 0,  1,  2,  3, 

Postbroadcast: on 3 process is 
 0,  1,  2,  3, 


**Question 3** 
Create a vector of size 8 and assign random values on the root process. Write
a funtion for Scatter of vector of size 2 from root process to the other processes.
<br/> MPI_Bcast sends the same piece of data to all processes while MPI_Scatter sends chunks of an array to different processes.

In [49]:
%%writefile ques3.c
#include<stdio.h>
#include<mpi.h>
#include<string.h>
#include<stdlib.h>
int main(int argc, char **argv)
{
 
int rank,size;
MPI_Init(&argc ,&argv ) ;
MPI_Comm_rank(MPI_COMM_WORLD,&rank) ; // get the rank of this process
MPI_Comm_size(MPI_COMM_WORLD,&size) ; //get the size of no of processors
int *a;
a=malloc(8*sizeof(int));
for(int i=0;i<8;i++)
{
a[i]=0;
}
if(rank==0)
{
for(int i=0;i<8;i++)
{
a[i]=i;
}
}
printf( "Pre−Broadcast: on %d process is \n" , rank ) ;
for(int i=0;i<8;i++)
{
printf(" %d, ",a[i]);
}
printf("\n");
MPI_Barrier(MPI_COMM_WORLD);
MPI_Scatter(a,2,MPI_INT,a,2,MPI_INT,0,MPI_COMM_WORLD);
printf("\n");
printf( "Postbroadcast: on %d process is \n" , rank ) ;
for(int i=0;i<8;i++)
{
printf(" %d, ",a[i]);
}
printf("\n");
MPI_Finalize();
free(a);
return 0;
}

Overwriting ques3.c


In [50]:
%%shell
mpicc ques3.c -o ques3
mpirun --allow-run-as-root -np 4 ./ques3

Pre−Broadcast: on 0 process is 
 0,  1,  2,  3,  4,  5,  6,  7, 
Pre−Broadcast: on 3 process is 
 0,  0,  0,  0,  0,  0,  0,  0, 
Pre−Broadcast: on 2 process is 
 0,  0,  0,  0,  0,  0,  0,  0, 
Pre−Broadcast: on 1 process is 
 0,  0,  0,  0,  0,  0,  0,  0, 

Postbroadcast: on 0 process is 
 0,  1,  2,  3,  4,  5,  6,  7, 

Postbroadcast: on 3 process is 
 6,  7,  0,  0,  0,  0,  0,  0, 

Postbroadcast: on 2 process is 
 4,  5,  0,  0,  0,  0,  0,  0, 

Postbroadcast: on 1 process is 
 2,  3,  0,  0,  0,  0,  0,  0, 


In [55]:
%%writefile ques3b.c
#include<stdio.h>
#include<mpi.h>
#include<stdlib.h>
int main(int argc,char **argv)
{
int rank,size;
MPI_Init(&argc,&argv);
MPI_Comm_rank(MPI_COMM_WORLD,&rank);
MPI_Comm_size(MPI_COMM_WORLD,&size);
int *a,*b;
a=malloc(8*sizeof(int));
b=malloc(8*sizeof(int));
for(int i=0;i<8;i++)
{
a[i]=0;
b[i]=0;
}
if(rank==0)
{
for(int i=0;i<8;i++)
{ 
a[i]=i;
}
}
printf("pre-broadcast on process %d:\n",rank);
for(int i=0;i<8;i++)
{
	printf("%d  ",a[i]);
}
printf("\n");
MPI_Barrier(MPI_COMM_WORLD);
MPI_Scatter(a,2,MPI_INT,b,2,MPI_INT,0,MPI_COMM_WORLD);
printf("\npost-broadcast on process %d:\n",rank);
for(int i=0;i<8;i++) 
{
printf("%d ",b[i]);
}
printf("\n")
free(a);
MPI_Finalize();
return 0;
}

Overwriting ques3b.c


In [56]:
%%shell
mpicc ques3b.c -o ques3b
mpirun --allow-run-as-root -np 4 ./ques3b

ques3b.c: In function ‘main’:
ques3b.c:39:1: error: expected ‘;’ before ‘free’
 free(a);
 ^~~~
pre-broadcast on process 1:
pre-broadcast on process 3:
0  0  0  0  0  0  0  0  
0  0  0  0  0  0  0  0  
pre-broadcast on process 2:
0  0  0  0  0  0  0  0  
pre-broadcast on process 0:
0  1  2  3  4  5  6  7  


post-broadcast on process 0:

post-broadcast on process 2:

post-broadcast on process 3:
post-broadcast on process 1:
2 3 0 0 0 0 0 0 4 5 0 0 0 0 0 0 6 7 0 0 0 0 0 0 0 1 0 0 0 0 0 0 

**Question 4**
Create a vector of size 4 and assign random values on all processes. Write a
funtion for Gather of vector of size 16 from all other processes to the root process.

In [66]:
%%writefile ques4.c
#include<stdio.h>
#include<mpi.h>
#include<stdlib.h>
#include<time.h>
int main(int argc,char** argv)
{
int rank,size,*a,*b;
a=malloc(4*sizeof(int));
MPI_Init(&argc,&argv);
MPI_Comm_rank(MPI_COMM_WORLD,&rank);
MPI_Comm_size(MPI_COMM_WORLD,&size);
printf("Pre gather array on process:%d\n",rank);
for(int i=0;i<4;i++)
{
a[i]=rand()%(20+rank);
printf("%d ",a[i]);
}
printf("\n");
MPI_Barrier(MPI_COMM_WORLD);
b=malloc(16*sizeof(int));
for(int i=0;i<16;i++)
{
	b[i]=0;
}
	
MPI_Gather(a,4,MPI_INT,b,4,MPI_INT,0,MPI_COMM_WORLD);
printf("post gather array of b on process %d is:",rank);
for(int i=0;i<16;i++)
	{ printf("%d ",b[i]);}
printf("\npost gather array a on process %d is:",rank);
for(int i=0;i<4;i++)
	{ printf("%d ",a[i]);}
printf("\n");
free(a);
free(b);
MPI_Finalize();
return 0;
}





Overwriting ques4.c


In [67]:
%%shell
mpicc ques4.c -o ques4
mpirun --allow-run-as-root -np 4 ./ques4

Pre gather array on process:2
Pre gather array on process:0
3 6 17 15 
17 10 17 13 
Pre gather array on process:1
1 4 9 19 
Pre gather array on process:3
11 0 6 2 
post gather array of b on process 3 is:0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
post gather array a on process 3 is:11 0 6 2 
post gather array of b on process 1 is:0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
post gather array a on process 1 is:1 4 9 19 
post gather array of b on process 2 is:0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
post gather array a on process 2 is:17 10 17 13 
post gather array of b on process 0 is:3 6 17 15 1 4 9 19 17 10 17 13 11 0 6 2 
post gather array a on process 0 is:3 6 17 15 


**Question 5**
Computing average of numbers (create 10,000 random numbers) with MPI_Scatter and MPI_Gather.
Here we create an array on the root rank and scatter it across all ranks. Compute the local average of the scattered array in each of the ranks and then sum it up using MPIReduce. 

In [45]:
%%writefile ques5.c
#include<stdio.h>
#include<mpi.h>
#include<stdlib.h>
#include<time.h>

double localaverage(int *b,int localn,int totaln)
{
double average=0;
for(int i=0;i<localn;i++)
{average+=b[i];}
average/=totaln;
return average;

}

int main(int argc,char ** argv)
{
int *a,*b,rank,size,localn,localbeg,localend,startindex;
double localav,totalav=0,actual_sum=0;
MPI_Init(&argc,&argv);
MPI_Comm_rank(MPI_COMM_WORLD,&rank);
MPI_Comm_size(MPI_COMM_WORLD,&size);

if(rank==0)
{
a=malloc(100*sizeof(int));
srand((time(0)));
for(int i=0;i<100;i++)
{
a[i]=rand()%100; 
actual_sum+=a[i];
}
actual_sum/=100;
printf("actual average is: %f\n",actual_sum);
}

localn=100/size;
b=malloc(localn*sizeof(int));
for(int i=0;i<localn;i++)
{
	b[i]=0;
}
MPI_Scatter(a,localn,MPI_INT,b,localn,MPI_INT,0,MPI_COMM_WORLD);
localav=localaverage(b,localn,100);
printf("localaverage on rank:%d is %f\n",rank,localav);
MPI_Barrier(MPI_COMM_WORLD);

MPI_Reduce(&localav,&totalav,1,MPI_DOUBLE,MPI_SUM,0,MPI_COMM_WORLD);

if(rank==0)
{
	printf("total average is:%f",totalav);
}

MPI_Finalize();
return 0;
}



Overwriting ques5.c


In [46]:
%%shell
mpicc ques5.c -o ques5
mpirun --allow-run-as-root -np 4 ./ques5

actual average is: 48.280000
localaverage on rank:0 is 14.810000
localaverage on rank:3 is 10.830000
localaverage on rank:2 is 13.410000
localaverage on rank:1 is 9.230000
total average is:48.280000

**Question 6**
Calculation of $\pi$ using Leibniz Formula, $\pi=4\cdot\sum_{i=0}^{n}\frac{(-1)^{i}}{2i+1}$.

In [94]:
%%writefile ques6.c
#include<stdio.h>
#include<mpi.h>
#include<stdlib.h>
#include<math.h>
int main(int argc,char** argv)
{
	double localsum=0,totalsum,tsum=1,n=100,k;
  int a,b,localn,rank,size;
	MPI_Init(&argc,&argv);
	MPI_Comm_rank(MPI_COMM_WORLD,&rank);
	MPI_Comm_size(MPI_COMM_WORLD,&size);
	localn=n/size;
	a=(localn*rank)+1;
	b=(localn)+(a-1);
  for(int i=a;i<=b;i++)
  {
    k=(double)1/((2*i)+1);
    if(i%2==0)
    {localsum+=k;}
    else
    {localsum-=k;}
  }
  printf("a is:%d,b is:%d,local sum of rank %d is:%f\n",a,b,rank,localsum);
  MPI_Reduce(&localsum,&totalsum,1,MPI_DOUBLE,MPI_SUM,0,MPI_COMM_WORLD);
  if(rank==0)
  {
    totalsum+=1;
    totalsum*=4;
    printf("totalsum is:%f",totalsum);
  }
  MPI_Finalize();
	return 0;
}

Overwriting ques6.c


In [95]:
%%shell
mpicc ques6.c -o ques6
mpirun --allow-run-as-root -np 4 ./ques6

a is:26,b is:50,local sum of rank 1 is:0.014513
a is:76,b is:100,local sum of rank 3 is:0.005765
a is:1,b is:25,local sum of rank 0 is:-0.224214
a is:51,b is:75,local sum of rank 2 is:-0.008191
totalsum is:3.151493

**Question 7**
Create a two random vectors of size 10,00,000 and computes the dot product ofthese two vectors. Display the CPU timings.

In [70]:
%%writefile ques7.c

#include<stdio.h>
#include<mpi.h>
#include<stdlib.h>
#include<time.h>

double locdot(int *a,int *b,int ln,double lprod)
{
	lprod=0;
	for(int i=0;i<ln;i++)
	{
		lprod+=a[i]*b[i];
	}
	return lprod;
}

int main(int argc,char ** argv)
{
int rank,size,*a,*b,*la,*lb,n=20,localn,process,sum=0;
double localdot,totaldot;
MPI_Status status;
MPI_Init(&argc,&argv);
MPI_Comm_rank(MPI_COMM_WORLD,&rank);
MPI_Comm_size(MPI_COMM_WORLD,&size);
if(rank==0)
{
  a=malloc(n*sizeof(int));
  b=malloc(n*sizeof(int));
  for(int i=0;i<n;i++)
  {
    a[i]=rand()%100;
    b[i]=rand()%100;
    sum+=(a[i]*b[i]);
  }
  printf("sequential sum is:%d\n",sum);
}
localn=n/size;
la=malloc(localn*sizeof(int));
lb=malloc(localn*sizeof(int));
MPI_Scatter(a,localn,MPI_INT,la,localn,MPI_INT,0,MPI_COMM_WORLD);
MPI_Scatter(b,localn,MPI_INT,lb,localn,MPI_INT,0,MPI_COMM_WORLD);
localdot=locdot(la,lb,localn,localdot);
MPI_Reduce(&localdot,&totaldot,1,MPI_DOUBLE,MPI_SUM,0,MPI_COMM_WORLD);
if(rank==0)
{
printf("totaldot product in parallel:%f",totaldot);
}
MPI_Finalize();
}

Overwriting ques7.c


In [71]:
%%shell
mpicc ques7.c -o ques7
mpirun --allow-run-as-root -np 4 ./ques7

sequential sum is:53292
totaldot product in parallel:53292.000000

**Question 8**
Create a random vector of size 100,000 integers and distribute it among 4 processors. Find the Minimum, Maximum and Sum of the numbers in the vector. Also,
find the sum of each local vector on that rank and find their product. Morover,
find the maximum and the minimum along thier localtion in the vector. Print the
CPU time of all the functions and observe the scalability over the processors.

In [76]:
%%writefile ques8.c
#include<stdio.h>
#include<mpi.h>
#include<stdlib.h>
#include<time.h>
int main(int argc,char** argv)
{
	int *a,*b,n=20,rank,size,localn,lsum=0,lmax,lmin,sum,max,min,s=0;
	MPI_Init(&argc,&argv);
	MPI_Comm_rank(MPI_COMM_WORLD,&rank);
	MPI_Comm_size(MPI_COMM_WORLD,&size);
	double t1,t2;
	MPI_Barrier(MPI_COMM_WORLD);
	t1=MPI_Wtime();
	if(rank==0)
	{
		a=malloc(20*sizeof(int));
		srand(time(0));
		for(int i=0;i<n;i++)
		{
			a[i]=rand()%100;
			s+=a[i];
			printf(" %d",a[i]);
		}
	}
	localn=n/size;
	b=malloc(localn*sizeof(int));
	MPI_Scatter(a,localn,MPI_INT,b,localn,MPI_INT,0,MPI_COMM_WORLD);
	lmax=b[0];
	lmin=b[0];
	lsum=0;
	for(int i=0;i<localn;i++)
	{
		if(b[i]>lmax)
		{lmax=b[i];}
	if(b[i]<lmin)
	{lmin=b[i];}
lsum+=b[i];
	}
	MPI_Reduce(&lmax,&max,1,MPI_INT,MPI_MAX,0,MPI_COMM_WORLD);
	MPI_Reduce(&lmin,&min,1,MPI_INT,MPI_MIN,0,MPI_COMM_WORLD);
	MPI_Reduce(&lsum,&sum,1,MPI_INT,MPI_SUM,0,MPI_COMM_WORLD);
	if(rank==0)
	{
		printf("\nmax:%d,min: %d,sum:%d \n",max,min,sum);
	}
	MPI_Barrier(MPI_COMM_WORLD);
	t2=MPI_Wtime();
	printf("computational time for rank %d is:%f\n",rank,t2-t1);
  MPI_Finalize();
	return 0;
}
	

Overwriting ques8.c


In [77]:
%%shell
mpicc ques8.c -o ques8
mpirun --allow-run-as-root -np 4 ./ques8

 7 11 79 98 26 94 57 9 56 2 82 33 32 34 24 16 22 36 51 52
max:98,min: 2,sum:821 
computational time for rank 0 is:0.000720
computational time for rank 2 is:0.000730
computational time for rank 1 is:0.000721
computational time for rank 3 is:0.000867
